In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import TA

def select_plan():
    conn = sqlite3.connect('database.db')
    df = pd.read_sql_query("SELECT user_id, strategy_name, stock_code, capital, strategy.* FROM plan, strategy where strategy_name = strategy.name", conn)
    conn.close()
    return df

def get_ticker(stock):
    conn = sqlite3.connect('database.db')
    stock_code = "'{0}'".format(stock)
    df = pd.read_sql_query("SELECT stock.code, date, open, high, low, close, adj_close, volume, board_lot FROM stock, ticker where stock.code = " + stock_code + " and stock.code = ticker.code", conn)
    conn.close()
    return df

df = select_plan()
for _, r in df.iterrows():
    rows = []
    for x in range(r.x_start, r.x_end+1, r.x_step):
        row = []
        for y in range(r.y_start, r.y_end+1, r.y_step):
            position, tx_df, trade_df, performance = TA.backtesting(r.stock_code, r.strategy_name, r.capital, get_ticker(r.stock_code), str(x), str(y))
            print(performance[9])
            row.append(performance[9])
        rows.append(row)
    print(rows)

In [7]:
import sys

import numpy as np

def convolve2D(image, kernel, padding=0, strides=1):
    # Cross Correlation
    kernel = np.flipud(np.fliplr(kernel))

    # Gather Shapes of Kernel + Image + Padding
    xKernShape = kernel.shape[0]
    yKernShape = kernel.shape[1]
    xImgShape = image.shape[0]
    yImgShape = image.shape[1]

    # Shape of Output Convolution
    xOutput = int(((xImgShape - xKernShape + 2 * padding) / strides) + 1)
    yOutput = int(((yImgShape - yKernShape + 2 * padding) / strides) + 1)
    output = np.zeros((xOutput, yOutput))

    # Apply Equal Padding to All Sides
    if padding != 0:
        imagePadded = np.zeros((image.shape[0] + padding*2, image.shape[1] + padding*2))
        imagePadded[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = image
    else:
        imagePadded = image

    # Iterate through image
    rows = []
    for x in range(image.shape[0]):
        # Exit Convolution
        if x > image.shape[0] - xKernShape:
            break
        # Only Convolve if y has gone down by the specified Strides
        if x % strides == 0:
            row = []
            for y in range(image.shape[1]):
            # Go to next row once kernel is out of bounds
                if y > image.shape[1] - yKernShape:
                    break
                try:
                    # Only Convolve if x has moved by the specified Strides
                    if y % strides == 0:
                        output[x, y] = (kernel * imagePadded[x: x + xKernShape, y: y + yKernShape]).mean()
                        row.append(output[x, y])
                except:
                    break
        rows.append(row)
    return rows

In [14]:
# Edge Detection Kernel
image = np.array([[0.0, 0.05762, 0.39164, 0.51195, 0.53947, 0.6099, 0.61694, 0.64319, 0.55794, 0.50668], [0.0688, 0.0, 0.561, 0.85301, 0.60633, 0.65773, 0.67307, 0.61793, 0.57017, 0.58891], [-0.04949, -0.17585, 0.0, 0.53133, 0.61169, 0.64516, 0.65347, 0.58652, 0.3228, 0.50744], [-0.06532, -0.20459, -0.08115, 0.0, 0.60195, 0.49254, 0.52612, 0.56598, 0.28565, 0.31771], [-0.09181, -0.13605, -0.13968, -0.14257, 0.0, 0.20694, 0.43965, 0.28969, 0.28837, 0.27616], [-0.1421, -0.14838, -0.1276, -0.04713, 0.05946, 0.0, 0.57572, 0.43081, 0.30436, 0.25785], [-0.13262, -0.15867, -0.12857, -0.03018, -0.00499, -0.12716, 0.0, 0.20029, 0.31102, 0.31155], [-0.14403, -0.10552, -0.11081, -0.04996, 0.07451, 0.01018, 0.15022, 0.0, 0.24197, 0.23054], [-0.12717, -0.11487, -0.02824, 0.03982, 0.04454, 0.0311, 0.02113, 0.04522, 0.0, 0.2477], [-0.05566, -0.09741, -0.02487, 0.07491, 0.13139, 0.15712, 0.02855, 0.19215, 0.13971, 0.0]])

# Edge Detection Kernel
kernel = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])

# Convolve and Save Output
output = convolve2D(image, kernel)
output = convolve2D(np.array(output), kernel)
output = convolve2D(np.array(output), kernel)
output = convolve2D(np.array(output), kernel)
output

[[0.1831801524157903, 0.26834907178783723],
 [0.09445364730986132, 0.17156959914647155]]